In [ ]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import time
from IPython.core.display import Pretty
from datetime import datetime

### 1. H-Net Job Guide 

 H-Net Job Guide URL Search Code

```
category_id = {1: African American History / Studies, 2: African History / Studies, 3: American History / Studies, 4: Ancient History, 5: Anthropology, 108: Arabic History / Studies, 6: Archaeology, 7: Architecture and Architectural History, 8: Archival Science, 9: Area Studies, 10: Art / Art History, 11: Asian American History / Studies, 12: Asian History / Studies, 13: Atlantic History / Studies, 14: Australian and New Zealand History / Studies, 15: Black History / Studies, 109: Borderlands, 16: British History / Studies, 17: Business History / Economic History / Studies, 18: Canadian History / Studies, 20: Chicana/o History / Studies, 19: Childhood and Education, 104: Chinese History / Studies, 21: Classical Studies, 22: Colonial and Post-Colonial History / Studies, 23: Communication, 113: Comparative Literature, 24: Composition, 25: Contemporary History, 118: Criminology, 26: Cultural History / Studies, 27: Dance and History of Dance, 28: Demographic History / Studies, 29: Digital Humanities, 30: Diplomacy and International Relations, 31: Early Modern History and Period Studies, 32: East Asian History / Studies, 106: Eastern Europe History / Studies, 34: Educational Technology, 35: Environmental History / Studies, 36: Ethnic History / Studies, 37: European History / Studies, 38: Film and Film History, 39: Fine Arts, 40: French History / Studies, 41: Geography, 42: German History / Studies, 43: Government and Public Service, 44: Health and Health Care, 45: Historic Preservation, 46: History Education, 47: History of Science, Medicine, and Technology, 48: Holocaust, Genocide, and Memory Studies, 49: Human Rights, 50: Humanities, 51: Immigration History / Studies, 116: Indigenous Studies, 52: Intellectual History, 53: Islamic History / Studies, 54: Italian History / Studies, 55: Japanese History / Studies, 56: Jewish History / Studies, 57: Journalism / Media Studies, 105: Korean History / Studies, 58: Labor History / Studies, 59: Languages, 61: Latin American and Caribbean History / Studies, 60: Law and Legal History, 62: Library and Information Science, 63: Linguistics, 64: Literature, 65: Local History, 67: Maritime History / Studies, 68: Medieval and Byzantine History / Studies, 69: Mexican History / Studies, 70: Middle East History / Studies, 71: Military History, 110: Modern European History / Studies, 72: Music and Music History, 73: Nationalism History / Studies, 74: Native American History / Studies, 75: Oral History, 76: Philosophy, 77: Political History / Studies, 78: Political Science, 80: Psychology, 81: Public Health, 82: Public History, 83: Public Policy, 117: Race / Ethnic Studies, 84: Religious Studies and Theology, 85: Research and Methodology, 86: Rhetoric, 87: Rural History / Studies, 107: Russian or Soviet History / Studies, 90: Social History / Studies, 91: Social Sciences, 102: Social Work, 92: Sociology, 93: South Asian History / Studies, 111: Southeast Asian History / Studies, 94: Spanish and Portuguese History / Studies, 100: Sport History / Studies, 95: Theatre and Performance History / Studies, 96: Urban Design and Planning, 97: Urban History / Studies, 101: Visual Studies, 98: Women, Gender, and Sexuality, 99: World History / Studies}
```




In [ ]:
#SELECT YOUR CATEGORY NUMBER IN "my_cat_list="
my_cat_list = ['10','11','12','104','29','32','38']

"""10: Art / Art History
11: Asian American History / Studies 
12: Asian History / Studies
104: Chinese History / Studies
113: Comparative Literature
29: Digital Humanities
32: East Asian History / Studies
38: Film and Film History
50: Humanities
64: Literature
"""

institution = []
position = []
link = []
deadline = []
discipline = []

for i in my_cat_list:
  hnet_URL = f"https://www.h-net.org/jobs/job_browse.php?category_id={i}"
  hnet_page = requests.get(hnet_URL)
  hnet_soup = BeautifulSoup(hnet_page.text, "html.parser")

#CHANGE THE JOB RELASE MONTH In 'string='
  if hnet_soup.find(name='p', string='July 2021'):
    month_post = hnet_soup.find(name='p', string='July 2021').next_sibling.next_sibling
  else: pass
  
  for a in month_post.find_all(name='a'):
    institution.append(a.parent.next_element.replace(", ", ""))
    position.append(a.text)
    link.append('https://www.h-net.org/jobs/' + a['href'])
  
  for i in link[len(deadline):]:
    post = requests.get(i)
    post_soup = BeautifulSoup(post.text, "html.parser")
    td1 = post_soup.find('td', string='Closing Date')
    date_str = td1.find_next().text
    date = datetime.strptime(date_str, '%m/%d/%Y')
    deadline.append(date)

    td2 = post_soup.find('td', string='Primary Category:')
    discipline.append(td2.find_next().text)

d = {'Institution':institution, 'Position': position, 'Deadline': deadline, 'Link': link,'Discipline': discipline}
df_hnet = pd.DataFrame(d)
df_hnet = df_hnet.drop_duplicates(subset=['Link'])
df_hnet

,Institution,Position,Deadline,Link,Discipline
0,Cornell University,2022-23 Society for the Humanities Fellowships...,2021-09-20,https://www.h-net.org/jobs/job_display.php?id=...,Humanities
1,Texas Tech University,Post-Doctoral Fellow in the Humanities: Anti-R...,2021-07-30,https://www.h-net.org/jobs/job_display.php?id=...,Humanities
2,German Historical Institute,Binational Visiting Tandem Fellowship in the H...,2021-11-02,https://www.h-net.org/jobs/job_display.php?id=...,Immigration History / Studies
3,Yale University,Assistant Professor in Qing History,2021-10-07,https://www.h-net.org/jobs/job_display.php?id=...,Asian History / Studies
4,Yale University,"Assistant Professor, Southeast Asia",2021-10-07,https://www.h-net.org/jobs/job_display.php?id=...,Asian History / Studies
5,Daniel K. InouyeAsia-Pacific Center for Securi...,Professor,2021-08-03,https://www.h-net.org/jobs/job_display.php?id=...,Asian History / Studies
9,University of Michigan,Assistant Professor/Postdoctoral Research Fellow,2021-09-15,https://www.h-net.org/jobs/job_display.php?id=...,Humanities
10,Rowan University,Teaching Fellowship in Digital History,2021-08-31,https://www.h-net.org/jobs/job_display.php?id=...,Digital Humanities
12,Singapore Management University,Full-Time Assistant Professor on the Education...,2021-08-27,https://www.h-net.org/jobs/job_display.php?id=...,Educational Technology


### 2. Inside Higher Education

Inside Higher Education URL Search Code (selected)


```
Keywords= *YOUR KEYWORDS*
FacultyJobs= {1: Arts & Humanities}
SpecialFilters= {513023:'Assistant Professor', 125:'Tenured & Tenure-Track', 126:'Adjunct', 127:'Postdoc', 128:'Alt-Ac'}
EmploymentType= {129:'Full Time', 130:'Part Time'}
InstitutionType={131:'Community College', 132:'Four-Year Institution'}
```







In [ ]:
#CHANGE THE KEYWORDS IN 'my_keyword='
my_keyword = ['chinese or asian','film or media','art or art history']

result = pd.DataFrame()

#CHANGE THE URL SEARCH CODE IN "ihe_URL=" IF NECCESSARY
for keyword in my_keyword:
  ihe_URL = f"https://careers.insidehighered.com/searchjobs/?Keywords={keyword}&radialtown=&LocationId=&RadialLocation=20&NearFacetsShown=true&CountryCode=&FacultyJobs=1&SpecialFilters=513023&SpecialFilters=125&SpecialFilters=127&EmploymentType=129"
  ihe_page = requests.get(ihe_URL)
  ihe_soup = BeautifulSoup(ihe_page.text, "html.parser")

  institution = []
  position = []
  link = []

#CHANGE THE SEARCH LIMIT IN "limit=" IF NECCESSARY
  for li in ihe_soup.find_all(name='li', attrs={'class':'lister__meta-item lister__meta-item--recruiter'}, limit=5)[3:]:
    institution.append(li.text)

  for h in ihe_soup.find_all(name='h3', attrs={'class':'lister__header'}, limit=5)[3:]:
    position.append(h.text)
    link.append('https://careers.insidehighered.com' + h.next['href'].strip())

  deadline = []

  for i in link:
    post = requests.get(i)
    post_soup = BeautifulSoup(post.text, "html.parser")
    td1 = post_soup.find('dt', string='Closing date')
    date_str = td1.find_next().text
    date = datetime.strptime(date_str, '%b %d, %Y')
    deadline.append(date)

  d = {'Institution':institution, 'Position': position, 'Deadline': deadline, 'Link': link}
  df_ihe = pd.DataFrame(d)
  df_ihe['Discipline'] = keyword
  result = result.append(df_ihe)

result

,Institution,Position,Deadline,Link,Discipline
0,Michigan State University,Assistant Professor-FixedTerm,2023-02-22,https://careers.insidehighered.com/job/2100275...,chinese or asian
1,University of Missouri-Kansas City,Director of Advanced Education in Periodontics...,2021-09-03,https://careers.insidehighered.com/job/2161376...,chinese or asian
0,University of Denver,"Visiting Teaching Assistant Professor, Media, ...",2021-08-13,https://careers.insidehighered.com/job/2167055...,film or media
1,Middle Georgia State University,10017846 Assistant Professor of Art,2021-09-28,https://careers.insidehighered.com/job/2177809...,film or media
0,Georgia College & State University,"Assistant Professor of Art: Art History, Moder...",2021-09-26,https://careers.insidehighered.com/job/2070651...,art or art history
1,Kennesaw State University,Limited Term Assistant Professor of Art History,2033-06-18,https://careers.insidehighered.com/job/2169250...,art or art history


### 3. Higher Ed Jobs


Higher Ed Jobs URL Search Code


```
JobCat={141:  Admin - Academic Advising, 42:  Admin - Administrative Assistants, 14:  Admin - Admissions and Enrollment, 219:  Admin - Adult and Continuing Education Programs, 15:  Admin - Alumni Relations, 36:  Admin - Arts and Museum Administration, 212:  Admin - Assessment,  Accreditation,  and Compliance, 16:  Admin - Athletics and Coaching, 17:  Admin - Broadcasting,  Radio,  and Television, 213:  Admin - Bursar and Student Accounts, 18:  Admin - Business and Financial Services, 22:  Admin - Career Development and Services, 177:  Admin - Child Care Services, 37:  Admin - Communications,  Marketing,  and Public Affairs, 172:  Admin - Conference and Events Administration, 21:  Admin - Counseling and Psychological Services, 217:  Admin - Curriculum Design, 23:  Admin - Development and Fundraising, 179:  Admin - Disability and Accessibility Services, 220:  Admin - Distance Education Programs, 35:  Admin - Diversity and Multicultural Affairs, 214:  Admin - Economic and Business Development, 205:  Admin - Executive Assistants, 25:  Admin - Extension and Outreach, 26:  Admin - Facilities Management, 148:  Admin - Faculty Development, 27:  Admin - Financial Aid, 143:  Admin - Food Services, 29:  Admin - Health and Medical Services, 30:  Admin - Human Resources, 211:  Admin - Information Technology - Database Administrator, 163:  Admin - Information Technology - IT Manager/Director, 161:  Admin - Information Technology - IT Support and Training, 175:  Admin - Information Technology - Multimedia, 173:  Admin - Information Technology - Network/System Administrator, 162:  Admin - Information Technology - Other IT, 159:  Admin - Information Technology - Programmer/Analyst, 160:  Admin - Information Technology - Web Developer, 31:  Admin - Institutional Research and Planning, 218:  Admin - Instructional Technology and Design, 32:  Admin - International Programs and Services, 150:  Admin - Laboratory and Research, 33:  Admin - Legal Affairs, 34:  Admin - Libraries, 206:  Admin - Occupational and Environmental Safety, 43:  Admin - Other Administrative Positions, 39:  Admin - Police and Public Safety, 24:  Admin - Publications and Editing, 149:  Admin - Registrars, 174:  Admin - Religious and Spiritual Life, 38:  Admin - Residence Life and Housing, 215:  Admin - Sales, 28:  Admin - Sponsored Programs,  Grants,  and Contracts, 40:  Admin - Student Affairs and Services, 216:  Admin - Tutors and Learning Resources, 41:  Admin - Volunteer Programs and Service Learning, 1:  Executive - Academic Vice Presidents and Provosts, 164:  Executive - Administrative Vice Presidents, 243:  Executive - Deans - Agriculture, 244:  Executive - Deans - Business, 245:  Executive - Deans - Communications, 246:  Executive - Deans - Education, 247:  Executive - Deans - Engineering, 248:  Executive - Deans - Fine and Applied Arts, 249:  Executive - Deans - Health, 250:  Executive - Deans - Law, 251:  Executive - Deans - Liberal Arts, 252:  Executive - Deans - Medicine, 254:  Executive - Deans - Other, 253:  Executive - Deans - Science, 207:  Executive - Other Executive, 4:  Executive - Presidents and Chancellors, 50:  Faculty - Agriculture - Agricultural Economics and Agribusiness, 51:  Faculty - Agriculture - Animal Science, 52:  Faculty - Agriculture - Entomology, 54:  Faculty - Agriculture - Environmental Science,  Ecology,  and Forestry, 140:  Faculty - Agriculture - Extension, 53:  Faculty - Agriculture - Food Science, 123:  Faculty - Agriculture - Horticulture and Landscape Architecture, 58:  Faculty - Agriculture - Other, 49:  Faculty - Agriculture - Plant and Soil Science, 57:  Faculty - Agriculture - Veterinary Medicine, 45:  Faculty - Business - Accounting, 256:  Faculty - Business - Business Administration, 228:  Faculty - Business - Business Law, 255:  Faculty - Business - Entrepreneurship, 44:  Faculty - Business - Finance, 183:  Faculty - Business - Hotel and Restaurant Management, 203:  Faculty - Business - Human Resources, 144:  Faculty - Business - Information Systems and Technology, 229:  Faculty - Business - International Business, 46:  Faculty - Business - Management, 47:  Faculty - Business - Marketing and Sales, 48:  Faculty - Business - Other Business, 130:  Faculty - Communications - Broadcast Journalism, 132:  Faculty - Communications - Journalism, 152:  Faculty - Communications - Media and Communication Studies, 133:  Faculty - Communications - Other Communications, 129:  Faculty - Communications - Public Relations and Advertising, 131:  Faculty - Communications - Radio,  Television,  and Film, 128:  Faculty - Communications - Speech and Public Speaking, 59:  Faculty - Education - Adult and Distance Education (study of), 62:  Faculty - Education - Counselor Education, 63:  Faculty - Education - Curriculum and Instruction, 64:  Faculty - Education - Edu. Admin and Leadership (study of), 226:  Faculty - Education - Educational Psychology, 68:  Faculty - Education - Higher Education (study of), 69:  Faculty - Education - Instructional Technology and Design, 75:  Faculty - Education - Other Education (study of), 166:  Faculty - Education - Reading and Literacy, 227:  Faculty - Education - School Psychology, 72:  Faculty - Education - Special Education, 184:  Faculty - Education - Teacher Education, 73:  Faculty - Education - Teacher Education - Early Childhood, 67:  Faculty - Education - Teacher Education - Elementary, 180:  Faculty - Education - Teacher Education - Middle School, 71:  Faculty - Education - Teacher Education - Secondary Education, 110:  Faculty - Engineering - Aerospace, 111:  Faculty - Engineering - Agricultural, 113:  Faculty - Engineering - Biological, 114:  Faculty - Engineering - Chemical, 115:  Faculty - Engineering - Civil and Environmental, 116:  Faculty - Engineering - Computer Engineering, 117:  Faculty - Engineering - Electrical, 146:  Faculty - Engineering - Industrial and Manufacturing, 118:  Faculty - Engineering - Mechanical, 120:  Faculty - Engineering - Other Engineering, 121:  Faculty - Fine and Applied Arts - Architecture, 126:  Faculty - Fine and Applied Arts - Art, 122:  Faculty - Fine and Applied Arts - Art History, 260:  Faculty - Fine and Applied Arts - Dance, 230:  Faculty - Fine and Applied Arts - Digital Arts, 197:  Faculty - Fine and Applied Arts - Fashion and Textile Design, 155:  Faculty - Fine and Applied Arts - Graphic Design, 201:  Faculty - Fine and Applied Arts - Industrial Design, 167:  Faculty - Fine and Applied Arts - Interior Design, 124:  Faculty - Fine and Applied Arts - Music, 127:  Faculty - Fine and Applied Arts - Other Fine and Applied Arts, 196:  Faculty - Fine and Applied Arts - Photography, 125:  Faculty - Fine and Applied Arts - Theatre, 151:  Faculty - Health - Communication Sciences and Disorders, 193:  Faculty - Health - Health Education and Promotion, 259:  Faculty - Health - Health Information Technology, 200:  Faculty - Health - Healthcare Administration, 185:  Faculty - Health - Nutrition and Dietetics, 138:  Faculty - Health - Other Health Faculty, 105:  Faculty - Health - Physical Education and Kinesiology, 153:  Faculty - Health - Physical and Occupational Therapy, 221:  Faculty - Health - Public and Environmental Health, 208:  Faculty - Health - Sports Mgmt,  Recreation,  and Leisure Studies, 134:  Faculty - Law and Legal Studies, 77:  Faculty - Liberal Arts - American Studies, 78:  Faculty - Liberal Arts - Anthropology, 156:  Faculty - Liberal Arts - Criminal Justice, 81:  Faculty - Liberal Arts - Economics, 82:  Faculty - Liberal Arts - English and Literature, 83:  Faculty - Liberal Arts - English as Second Language, 76:  Faculty - Liberal Arts - Ethnic and Multicultural Studies, 157:  Faculty - Liberal Arts - Foreign Languages and Literatures, 139:  Faculty - Liberal Arts - History, 199:  Faculty - Liberal Arts - Human Development and Family Studies, 204:  Faculty - Liberal Arts - Humanities, 88:  Faculty - Liberal Arts - Linguistics, 97:  Faculty - Liberal Arts - Other Liberal Arts, 89:  Faculty - Liberal Arts - Philosophy, 90:  Faculty - Liberal Arts - Political Science, 91:  Faculty - Liberal Arts - Psychology, 209:  Faculty - Liberal Arts - Public Administration and Policy, 92:  Faculty - Liberal Arts - Religious Studies and Theology, 222:  Faculty - Liberal Arts - Security Studies, 261:  Faculty - Liberal Arts - Sign Language, 169:  Faculty - Liberal Arts - Social Work, 93:  Faculty - Liberal Arts - Sociology, 202:  Faculty - Liberal Arts - Urban Studies and Planning, 96:  Faculty - Liberal Arts - Women's and Gender Studies, 181:  Faculty - Medicine - Dentistry, 192:  Faculty - Medicine - Emergency Medical Services, 137:  Faculty - Medicine - Medical Researcher, 136:  Faculty - Medicine - Nursing, 210:  Faculty - Medicine - Other Medicine, 154:  Faculty - Medicine - Pharmacology, 223:  Faculty - Medicine - Physician Assistant, 135:  Faculty - Medicine - Physicians, 168:  Faculty - Medicine - Radiology, 98:  Faculty - Science - Astronomy and Astrophysics, 99:  Faculty - Science - Biochemistry and Molecular Biology, 100:  Faculty - Science - Biology, 101:  Faculty - Science - Chemistry, 102:  Faculty - Science - Computer Science, 103:  Faculty - Science - Geography, 170:  Faculty - Science - Geology,  Earth Sciences,  and Oceanography, 182:  Faculty - Science - Library and Information Science, 104:  Faculty - Science - Mathematics, 108:  Faculty - Science - Other Science, 106:  Faculty - Science - Physics, 107:  Faculty - Science - Statistics, 190:  Faculty - Vocational and Technical - Automotive Technology, 187:  Faculty - Vocational and Technical - Aviation, 189:  Faculty - Vocational and Technical - Construction and Building Trades, 224:  Faculty - Vocational and Technical - Cosmetology, 188:  Faculty - Vocational and Technical - Culinary Arts, 195:  Faculty - Vocational and Technical - Electronics, 198:  Faculty - Vocational and Technical - Fire Science, 225:  Faculty - Vocational and Technical - Massage Therapy, 257:  Faculty - Vocational and Technical - Medical Assistants, 258:  Faculty - Vocational and Technical - Medical Billing and Coding, 74:  Faculty - Vocational and Technical - Other, 191:  Faculty - Vocational and Technical - Telecommunications, 186:  Faculty - Vocational and Technical - Tourism}
PosType={1: Full-Time, 2: Adjunct/Part-Time}
InstType={1: Four-Year/Advanced Degree (B.A., M.A., and/or Ph.D.), 2: Two-Year/Community College (Associate Degree), 3: Outside Higher Education}
Remote={1: Location, 2: Remote}
```


In [ ]:
# server checks headers & change configuration to bypass the blockage

def get_page_source(n):
    url = 'https://www.higheredjobs.com/search/advanced_action.cfm?JobCat=152&JobCat=131&JobCat=82&JobCat=76&JobCat=157&JobCat=204&JobCat=97&PosType=1&InstType=1&Keyword=chinese+or+asian&Remote=1&Region=&Submit=Search+Jobs'
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    response = requests.get(url, headers=headers)
    return response.text

n = 1006233
text = get_page_source(n)

### 5. Indeed

In [ ]:
https://www.indeed.com/jobs?q=chinese%20professor&jt=fulltime

### 4. HERC

In [ ]:
# Server not correctly configured, & certificate not verified
herc_URL = "https://main.hercjobs.org/jobs?keywords=chinese+or+asian&place=all&discipline=academic-faculty"
herc_page = requests.get(herc_URL)
herc_soup = BeautifulSoup(herc_page.text, "html.parser")
herc_soup